#Data Preprocessing

Published Documents:
 - [Spotify Podcasts Dataset](https://arxiv.org/pdf/2004.04270v3.pdf)
 - [Speech Recognition Diarization](https://arxiv.org/pdf/2005.08072.pdf)
 - 


Dataset:
 - [This American Life Podcast Transcripts](https://www.kaggle.com/datasets/shuyangli94/this-american-life-podcast-transcriptsalignments?resource=download)

Citation:

 - Mao, H. H., Li, S., McAuley, J., & Cottrell, G. (2020). Speech Recognition and Multi-Speaker Diarization of Long Conversations. INTERSPEECH.

## Imported Packages and Libraries

In [15]:
!pip install nlp --quiet

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 4254, in parseImpl
    ret = e._parse(instring, loc, doActions)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 1687, in _parseNoCache
    loc, tokens = self.parseImpl(instring, preloc, doActions)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 2893, in parseImpl
    raise ParseException(instring, loc, self.errmsg, self)
pip._vendor.pyparsing.ParseException: Expected "implementation_versio

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow import keras
from nlp import load_dataset

import seaborn as sns
from pprint import pprint 

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json

# Formatting options for float number in numpy
float_formatter = "{:.4f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

## The American Life Podcast Dataset

### Load

In [19]:
dataset_path = '/content/drive/MyDrive/MIDS/W266/w266_nlp_project/TALDataset'

In [20]:
full_speaker_data = 'full-speaker-map.json'
test_transcript_data = 'test-transcripts-aligned.json'
train_transcript_data = 'train-transcripts-aligned.json'
valid_transcript_data = 'valid-transcripts-aligned.json'

In [21]:
ep_num = 11

In [22]:
path_train = os.path.join(dataset_path,train_transcript_data)
f = open(path_train,)
train_dataset = json.load(f)

In [23]:
pprint(train_dataset.get('ep-1')[10])

{'act': 'prologue',
 'act_title': 'Prologue',
 'alignments': [[33.14, 34.14, 0], [34.14, 39.94, 1]],
 'duration': 6.82,
 'ends_q': False,
 'episode': 'ep-1',
 'has_q': False,
 'n_sentences': 1,
 'n_words': 4,
 'role': 'interviewer',
 'speaker': 'ira glass',
 'utterance': "That's right.",
 'utterance_end': 39.96,
 'utterance_start': 33.14}


### Process

In [44]:
#train_dataset.keys()
S = []            #  list of M utterances S = {S_1,..., S_M}
T = []            #  Underlying topic structure Ti ∈ [Sj , Sk].
Y = []            #  Label sequence Y = {y1,.., yM} yi is binary indicates whether the utterance Si is the start of a new topic segment
topic_label = []  #  Topic label
speaker = []      #  Speaker as in  epsiode.role
speaker_name = [] #  Speaker name as in episode.speaker
episode = []      #  Episode Number
alignments_T = [] #  Alignments correspondant to topic structure

topic_1 = ""  
idx_1 = 0    
utterance_start_1 = 0
for idx, i in enumerate(train_dataset['ep-1']):
  S.append(i['utterance'])
  topic = i['act']
  topic_label.append(topic)
  speaker.append(i['role'])
  speaker_name.append(i['speaker'])
  episode.append(i['episode'])
  if topic != topic_1:
    T.append((idx_1,idx))
    alignments_T.append((utterance_start_1,i['utterance_start']))
    Y.append(1)
    idx_1 = idx
    utterance_start_1 = i['utterance_start']
  else:
    Y.append(0)
  topic_1 = topic


#train_dataset['ep-1'][0]
#train_dataset['ep-1'][0]['role']
#pprint(train_dataset.get('ep-1')[10])
print(Y)
print(S)
print(T)
print(topic_label)
print(speaker)
print(speaker_name)
print(episode)
print(alignments_T)


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
['Joe Franklin?', "I'm ready.", "It's Ira Glass here.", "Oh, you're the emcee on the show, Ira.", 'I am the emcee on the show. Yes.', 'Oh great. Ira? I-R-A, Ira?', 'Ira, I-R-A.', "Oh, great. Now hold on one second, Ira. Don't go away. Hello? [UNINTELLIGIBLE]. Call me after 3 o'clock. I have great news for you. Ira.", 'Yes.', "So listen, Tony. If the phone rings, take it in the back. And then come out and tell me who it is. Just 

## SYSK Dataset

In [25]:
f = open('/content/drive/MyDrive/W266 Final Project/datasets/SYSK/sysk_with_transcripts_10 Accidental Inventions By the Numbers.json')

sysk_train_dataset = json.load(f)

FileNotFoundError: ignored

In [ ]:
pprint(sysk_train_dataset.get('text'))